In [1]:
import pandas as pd
from portfolio_analysis.dcf import get_irr, get_irr_new, get_dividend_ratio
from portfolio_analysis.data import get_single_company_data, get_all_company_tickers
from portfolio_analysis.api import apikey

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
company_names = get_all_company_tickers(apikey)

In [4]:
mf_tickers = pd.read_csv('/Users/murtazavahanvaty/OneDrive - Cornerstone Capabilities/Personal/portfolio-analysis/excels/MF_500MM_50_NOV6.csv')

In [8]:
mf_company_names = company_names[company_names.symbol.isin(mf_tickers.TICKERS.unique())]
mf_annual = pd.DataFrame()
for index, company in mf_company_names.iterrows():
    symbol = company.loc['symbol']
    company_data = get_single_company_data(symbol, apikey)
    # try:
    irr = get_irr(company, company_data["IS"], company_data["Profile"], company_data["BS"], company_data["MC"], company_data["CFR"], company_data["CFS"])
    # except KeyError as e:
    #     failed_companies = failed_companies.append(company, ignore_index=True)
    #     print(e)
    #     continue
    mf_annual = mf_annual.append(irr, ignore_index=True)

/Users/murtazavahanvaty/.local/share/virtualenvs/portfolio-analysis-r1h2Z-bq/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/murtazavahanvaty/.local/share/virtualenvs/portfolio-analysis-r1h2Z-bq/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/murtazavahanvaty/.local/share/virtualenvs/portfolio-analysis-r1h2Z-bq/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/murtazavahanvaty/.local/share/virtualenvs/portfolio-analysis-r1h2Z-bq/li

In [9]:
mf_quarter = pd.DataFrame()
for index, company in mf_company_names.iterrows():
    symbol = company.loc['symbol']
    company_data = get_single_company_data(symbol, apikey, period='quarter')
    # try:
    irr = get_irr_new(company, company_data["IS"], company_data["Profile"], company_data["BS"], company_data["MC"], company_data["CFR"], company_data["CFS"])
    # except KeyError as e:
    #     failed_companies = failed_companies.append(company, ignore_index=True)
    #     print(e)
    #     continue
    mf_quarter = mf_quarter.append(irr, ignore_index=True)

/Users/murtazavahanvaty/.local/share/virtualenvs/portfolio-analysis-r1h2Z-bq/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/murtazavahanvaty/.local/share/virtualenvs/portfolio-analysis-r1h2Z-bq/lib/python3.8/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/murtazavahanvaty/.local/share/virtualenvs/portfolio-analysis-r1h2Z-bq/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/murtazavahanvaty/.local/share/virtualenvs/portfolio-analysis-r1h2Z-bq/l

In [12]:
compare_irr = mf_annual.merge(mf_quarter[['symbol','income_statement_date','ROC','EarningsYield','irr', 'npv_mean', 'npv_regression', 'regression_type',
       'eps_roc', 'eps_roc_regression', 'ROE', 'eps_diluted', 'MOP', 'QA',
       'MCap', 'PE', 'eps_roc_flag', 'eps_base', 'eps_list', 'dividend_ratio',
       'dividend_trend', 'commonstock_repurchased_trend',
       'commonstock_repurchased']],on='symbol',suffixes=('','_quarter'))

In [13]:
columns = ['symbol', 'name', 'price', 'industry', 'error_message', 'reportedCurrency']
for col in mf_annual.columns:
    if col not in ['symbol', 'name', 'price', 'industry', 'error_message', 'reportedCurrency']:
        columns+=[col, f'{col}_quarter']

In [ ]:
# Merge in sector

In [17]:
sector_df = pd.read_csv("company_sectors.csv")

In [20]:
mf_final = compare_irr[columns].merge(sector_df[["symbol","sector"]], on = 'symbol')

In [22]:
mf_final.to_csv('mf_company_analysis.csv')